In [1]:
import math
import os

import numpy as np
import pandas as pd

In [ ]:
import re

In [ ]:
def re_filter(text):
    expression = '''drink drinker drinks drinking drank wine champgne alcohol alcoholics alcoholism beer beers bottle bottles
    pint pints cocktail cocktails bar brewery lounge pub liquor booze vodka tequila gin ciroc margarita margaritas shot shots
    ale whiskey lager tipsy drunk sober wasted pregame pregaming'''.split().join('|')
    alcohol_filter = re.compile(expression)
    return bool(alcohol_filter.match(text))

In [10]:
keywords = set('''drink drinker drinks drinking drank wine champgne alcohol alcoholics alcoholism beer beers bottle bottles
    pint pints cocktail cocktails bar brewery lounge pub liquor booze vodka tequila gin ciroc margarita margaritas shot shots
    ale whiskey lager tipsy drunk sober wasted pregame pregaming'''.split())

def prefilter(text):
    return bool(set(text.split()).intersection(keywords))

In [18]:
df = pd.read_csv('E:/september blocked/remaining_tweets_0.csv')
df2 = pd.read_csv('E:/september untarred/twitter_2015-09-01_00h.json.tar.gz0.csv')

C:\Users\Tom Work\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (0,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
df2.head()

,text,id,created_at
0,Everyone going to philly don't rush it it's go...,638561918433759232,Tue Sep 01 03:59:59 +0000 2015
1,If she ain't tryin get it she can get dropped ...,638561918148440064,Tue Sep 01 03:59:59 +0000 2015
2,Was i the only one who thought gohan was cute?...,638561918123270144,Tue Sep 01 03:59:59 +0000 2015
3,@TungguSekejap bila doh!? Aku kat ny skunk. Huhu,638561918618312704,Tue Sep 01 03:59:59 +0000 2015
4,@IamHolleman we still need to hit up that new ...,638561918660288512,Tue Sep 01 03:59:59 +0000 2015


In [29]:
df.to_csv('E:/test.csv', encoding='utf8')

Split uniformly by number of tweets instead of by time

In [3]:
def block(in_folder, out_folder, block_size):
    tweets = pd.DataFrame()
    file_num = 0
    for file in os.listdir(in_folder):
        _df = pd.read_csv(in_folder + '/'  + file)[['id', 'created_at', 'utc_offset', 'text']]
        tweets = pd.concat([tweets, _df])
        while len(tweets) > block_size:
            tweets[:block_size].to_csv(out_folder + '/tweets_' + str(file_num) +'.csv', encoding='utf8')
            tweets = tweets[block_size:]
            file_num += 1
    tweets.to_csv(out_folder + '/tweets_' + str(file_num) +'.csv')

In [7]:
%%time
block('E:/Sept-March 2015/csv with offset/sept', 'E:/Sept-March 2015/csv with offset/sept blocked', 100000)

Wall time: 30min 19s


In [3]:
%%time
block('E:/Sept-March 2015/csv with offset/march', 'E:/Sept-March 2015/csv with offset/march blocked', 1000000)

Wall time: 46min 6s


In [ ]:
block('E:/Sept-March 2015/csv with offset/june', 'E:/Sept-March 2015/csv with offset/june blocked', 1000000)

In [4]:

block('E:/Sept-March 2015/csv with offset/dec', 'E:/Sept-March 2015/csv with offset/dec blocked', 1000000)

In [6]:
%%time
sample('E:/Sept-March 2015/csv with offset/march blocked', 'E:/Sept-March 2015/csv with offset/march sample', 500000, 0.03)


tweets_52.csv
'utf-8' codec can't decode byte 0x92 in position 2954: invalid start byte


KeyboardInterrupt: 

In [19]:
%%time
sample('E:/Sept-March 2015/csv with offset/june blocked', 'E:/Sept-March 2015/csv with offset/june sample', 500000, 0.03)
sample('E:/Sept-March 2015/csv with offset/dec blocked', 'E:/Sept-March 2015/csv with offset/dec sample', 500000, 0.03)
sample('E:/Sept-March 2015/csv with offset/sept', 'E:/Sept-March 2015/csv with offset/sept sample', 500000, 0.03)

tweets_67.csv
'utf-8' codec can't decode byte 0x85 in position 513: invalid start byte
tweets_26.csv
'utf-8' codec can't decode byte 0x96 in position 2240: invalid start byte
remaining
[Errno 13] Permission denied: 'E:/Sept-March 2015/csv with offset/sept/remaining'
Wall time: 1h 9min 3s


In [28]:
sample('E:/Sept-March 2015/csv with offset/sept', 'E:/Sept-March 2015/csv with offset/sept sample/15 percent', 500000, 0.15)

In [17]:
%%time
def label(in_folder):
    i = 0
    for file in  os.listdir(in_folder):
        sample = pd.read_csv(in_folder + '/' + file,  engine='python').dropna()
        labeled = clf(sample)
        labeled.index = pd.to_datetime(labeled.created_at)
        pickle.dump(labeled, open(in_folder +'/drinking_labeled_' + str(i) + '.p', 'wb'))
        i += 1

Wall time: 0 ns


In [16]:
%%time
label('E:/Sept-March 2015/csv with offset/march sample')

ValueError: The 'dtype' option is not supported with the 'python' engine

In [29]:
%%time
label('E:/Sept-March 2015/csv with offset/sept sample/15 percent')

Wall time: 7h 36min 6s


In [21]:
%%time
label('E:/Sept-March 2015/csv with offset/june sample')

Wall time: 1h 36min 44s


In [22]:
%%time
label('E:/Sept-March 2015/csv with offset/dec sample')

Wall time: 32min 48s


In [ ]:
block('E:/Sept-March 2015/csv with offset/sept', 'E:/Sept-March 2015/csv with offset/sept blocked', 1000000)

Select a random sample %

In [5]:
def sample(in_folder, out_folder, block_size=200000, fraction=0.05):
    tweets = pd.DataFrame()
    file_num = 0
    for file in os.listdir(in_folder):
        try:
            _df = pd.read_csv(in_folder + '/'  + file, encoding='utf8', engine='python')
            _df_sample = _df.sample(int(len(_df)*fraction))
            _df['id'].apply(str)
            tweets = pd.concat([tweets, _df_sample])
            
            if len(tweets) > block_size:
                tweets[:block_size].to_csv(out_folder + '/random_' + str(file_num) +'.csv', encoding='utf8')
                tweets = tweets[block_size:]
                file_num += 1
        except Exception as e:
            print (file)
            print (e)
    tweets.to_csv(out_folder + '/tweets_' + str(file_num) +'.csv')

In [15]:
sample('E:/sept-march 2015/march blocked', 'E:/sept-march 2015/sample')

tweets_104.csv
'utf-8' codec can't decode byte 0xf1 in position 573: invalid continuation byte


KeyboardInterrupt: 

Split into drinking an nondrinking by keyword

In [42]:
def seperate_drinking(in_folder, out_folder, block_size):
    drinking = pd.DataFrame()
    i = 0
    for file in os.listdir(in_folder):
        try:
            _df = pd.read_csv(in_folder + '/' + file, encoding='utf8', engine='python')
            drinking = pd.concat([drinking, _df[_df.text.apply(str).apply(prefilter)]])
            drinking['id'].apply(str)
            if len(drinking) > block_size:
                drinking[:block_size].to_csv(out_folder + '/drinking_' + str(i) +'.csv', encoding='utf8')
                drinking = drinking[block_size:]
                i+=1
        except Exception as e:
            print (file)
            print (e)
    drinking.to_csv(out_folder + '/drinking_' + str(i) +'.csv', encoding='utf8')

In [21]:
drinking = pd.read_csv('E:/september blocked/drinking/drinking_0.csv', dtype={'id': str})

In [13]:
drinking.shape

(498303, 5)

In [23]:
drinking['random_number'] = drinking.text.apply(hash)

In [22]:
(drinking.id[0])

'638561918660288512'

In [24]:
drinking.index = drinking.id

In [26]:
drinking.head()

,Unnamed: 0,Unnamed: 0.1,text,id,created_at,random_number
id,,,,,,
638561918660288512,4,4,@IamHolleman we still need to hit up that new ...,638561918660288512,Tue Sep 01 03:59:59 +0000 2015,4020266428001935890
638561921806024704,39,39,@young_zamp we aren't drinking lol,638561921806024704,Tue Sep 01 04:00:00 +0000 2015,3871613784586990036
638561941477322753,255,255,@Janaeethomas idk if it works or not but I scr...,638561941477322753,Tue Sep 01 04:00:05 +0000 2015,-4684982304355642440
638561967897231360,537,537,I'm feeling tipsy and not in the good way,638561967897231360,Tue Sep 01 04:00:11 +0000 2015,5214561322629894300
638562056208183296,1491,1491,wine gone knock me out everytime 😅 I been slee...,638562056208183296,Tue Sep 01 04:00:32 +0000 2015,1935074228051928726


In [27]:
pickle.dump(drinking, open('pickles/sept_drinking.p', 'wb'))

In [28]:
import os
import datetime
def make_batch(pickled_df, size, output_dir):
    """
    pickled_df: unlabeled data with 'random_number' column
    size: # of items batch to be created
    output_dir: where the outputs are saved
    """
    # Load data
    X = pickle.load(open(pickled_df, 'rb'))
    
    # select size subset
    batch = X.sort_values(by='random_number').head(size)
    
    # process for turk
    batch["text"] = batch.text.str.encode("utf-8")
    date = str(datetime.date.today())
    time = datetime.datetime.now().time()
    key = batch.iloc[0].random_number
    
    # persist
    description = "/c_amt {} {}h{}m {}".format(date, time.hour, time.minute, key)
    if not os.path.exists(output_dir + description):
        os.makedirs(output_dir + description)
    batch[["text", "random_number"]].to_csv(output_dir + description +  description + '.csv')
    pickle.dump(batch, open(output_dir + description + description +".p", 'wb'))
    X = X.drop(batch.index)
    pickle.dump(X, open(pickled_df, 'wb'))

In [29]:
make_batch('pickles/sept_drinking.p', 1000, 'amt/sept')

In [7]:
import pickle
from classification.prediction import PredictionTransformer

In [8]:
clf_alc = pickle.load(open('pickles/clf_alc_UPDATED.p', 'rb'))
clf_fpa = pickle.load(open('pickles/clf_fpa_UPDATED.p', 'rb'))
clf_fpl = pickle.load(open('pickles/clf_fpl_double_labeled', 'rb'))
clf = PredictionTransformer(clf_alc, clf_fpa, clf_fpl)

In [35]:
%%time
labeled = clf(drinking)

Wall time: 2h 8min 23s


In [36]:
%%time
labeled.index = pd.to_datetime(labeled.created_at)

Wall time: 5min 3s


In [38]:
pickle.dump(labeled, open('pickles/sept_drinking_labeled.p', 'wb'))

In [43]:
%%time
block('E:/march untarred', 'E:/march blocked', 500000)

Wall time: 19min 19s


In [44]:
%%time
block('E:/june untarred', 'E:/june blocked', 500000)

Wall time: 24min 16s


In [21]:
%%time
sample('E:/sept-march 2015/june blocked', 'E:/sept-march 2015/sample/june')

tweets_134.csv
'utf-8' codec can't decode byte 0x85 in position 310: invalid start byte
Wall time: 22min 37s


In [22]:
%%time
sample('E:/sept-march 2015/march blocked', 'E:/sept-march 2015/sample/march')

tweets_104.csv
'utf-8' codec can't decode byte 0xf1 in position 573: invalid continuation byte
Wall time: 17min 4s


In [24]:
%%time
sample('E:/sept-march 2015/september blocked', 'E:/sept-march 2015/sample/sept')

remaining_tweets_136.csv
'utf-8' codec can't decode byte 0x85 in position 395: invalid start byte
Wall time: 19min 23s


In [33]:
%%time
in_folder = 'E:/sept-march 2015/sample/june'
i = 0
for file in os.listdir(in_folder):
    june_sample = pd.read_csv(in_folder + '/' + file, dtype={'id': str}).dropna()
    june_labeled = clf(june_sample)
    june_labeled.index = pd.to_datetime(june_labeled.created_at)
    pickle.dump(june_labeled, open('E:/sept-march 2015/sample/june_new_drinking_labeled_' + str(i) + '.p', 'wb'))
    i += 1

<string>:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x85 in position 71: invalid start byte

In [6]:
%%time
in_folder = 'E:/sept-march 2015/sample/march'
i = 0
for file in  os.listdir(in_folder):
    june_sample = pd.read_csv(in_folder + '/' + file, dtype={'id': str}).dropna()
    june_labeled = clf(june_sample)
    june_labeled.index = pd.to_datetime(june_labeled.created_at)
    pickle.dump(june_labeled, open('E:/sept-march 2015/sample/march_new_drinking_labeled_' + str(i) + '.p', 'wb'))
    i += 1

Wall time: 3h 5min 29s


<string>:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.


In [7]:
%%time
in_folder = 'E:/sept-march 2015/sample/sept'
i = 0
for file in  os.listdir(in_folder):
    june_sample = pd.read_csv(in_folder + '/' + file, dtype={'id': str}).dropna()
    june_labeled = clf(june_sample)
    june_labeled.index = pd.to_datetime(june_labeled.created_at)
    pickle.dump(june_labeled, open('E:/sept-march 2015/sample/sept_new_drinking_labeled_' + str(i) + '.p', 'wb'))
    i += 1

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x85 in position 92: invalid start byte